In [0]:
import random
import collections
import math
import os
import zipfile
import time
import re
import numpy as np
import tensorflow as tf 
from matplotlib import pylab
%matplotlib inline
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [0]:
dataset_link = 'http://mattmahoney.net/dc/'
zip_file = 'text8.zip'

In [0]:
def data_download(zip_file):

  if not os.path.exists(zip_file):
    zip_file, _ = urlretrieve(dataset_link + zip_file, zip_file)
    print('File downloaded successfully!')
    
    return None

In [0]:
data_download(zip_file)

In [0]:
!ls

dataset  sample_data  text8.zip


In [0]:
extracted_folder = 'dataset'
if not os.path.isdir(extracted_folder):
  
  with zipfile.ZipFile(zip_file) as zf:
    zf.extractall(extracted_folder)
    
with open('dataset/text8') as ft_ :
  full_text = ft_.read()

In [0]:
def text_preprocessing(text):
  text=text.lower()
  text=text.replace('.','<period>')
  text=text.replace(',','<comma>')
  text=text.replace('"','<quotation>')
  text = text.replace(';', ' <semicolon> ')
  text = text.replace('!', ' <exclamation> ')
  text = text.replace('?', ' <question> ')
  text = text.replace('(', ' <paren_l> ')
  text = text.replace(')', ' <paren_r> ')
  text = text.replace('--', ' <hyphen> ')
  text = text.replace(':', ' <colon> ')
  text_tokens = text.split()
  return text_tokens

In [0]:
full_text[:200]

' anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term '

In [0]:
text_tokens=text_preprocessing(full_text)

In [0]:

text_tokens[:100]

['anarchism',
 'originated',
 'as',
 'a',
 'term',
 'of',
 'abuse',
 'first',
 'used',
 'against',
 'early',
 'working',
 'class',
 'radicals',
 'including',
 'the',
 'diggers',
 'of',
 'the',
 'english',
 'revolution',
 'and',
 'the',
 'sans',
 'culottes',
 'of',
 'the',
 'french',
 'revolution',
 'whilst',
 'the',
 'term',
 'is',
 'still',
 'used',
 'in',
 'a',
 'pejorative',
 'way',
 'to',
 'describe',
 'any',
 'act',
 'that',
 'used',
 'violent',
 'means',
 'to',
 'destroy',
 'the',
 'organization',
 'of',
 'society',
 'it',
 'has',
 'also',
 'been',
 'taken',
 'up',
 'as',
 'a',
 'positive',
 'label',
 'by',
 'self',
 'defined',
 'anarchists',
 'the',
 'word',
 'anarchism',
 'is',
 'derived',
 'from',
 'the',
 'greek',
 'without',
 'archons',
 'ruler',
 'chief',
 'king',
 'anarchism',
 'as',
 'a',
 'political',
 'philosophy',
 'is',
 'the',
 'belief',
 'that',
 'rulers',
 'are',
 'unnecessary',
 'and',
 'should',
 'be',
 'abolished',
 'although',
 'there',
 'are',
 'differing']

In [0]:
len(text_tokens)

17005207

In [0]:
len(text_tokens)

17005207

In [0]:
word_count=collections.Counter(text_tokens)

In [0]:
len(word_count)

253854

In [0]:
sorted(word_count,key=word_count.get,reverse=True)[:5]  # as we can see stop words comes first

['the', 'of', 'and', 'one', 'in']

In [0]:
#Removing words with count <7      # Sub Sampling 
shortlisted_words=[w for w in text_tokens if word_count[w]>7]

In [0]:
len(word_count)

253854

In [0]:
len(shortlisted_words)

16616688

In [0]:
sorted(word_count,key=word_count.get,reverse=True)[:5]

['the', 'of', 'and', 'one', 'in']

In [0]:
## To process the unique words present in the corpus, we have made a
# set of the words, followed by their frequency in the training dataset. The
# following function creates a dictionary and converts words to integers
# and, conversely, integers to words. The most frequent word is assigned the
# least value, 0, and in similar fashion, numbers are assigned to other words.
# Conversion of words to integers has been stored in a separate list.###

In [0]:
word2index={x[0]:j for j,x in enumerate(sorted(word_count.items(),key=lambda x:x[1],reverse=True)[:5000])}

In [0]:
len(word2index)

5000

In [0]:
len(word_count)

253854

In [0]:
a={x[0]:x[1] for j,x in enumerate(sorted(word_count.items(),key=lambda x:x[1],reverse=True)[:5000])}

In [0]:
sum([a[i] for i in a])/len(shortlisted_words) #85 % of  data is present

0.8587794390795567

In [0]:
# making new dictionary of new shortlisted words

def dict_create(shotlisted_words):
  counts1=collections.Counter(shotlisted_words)
  vocabulary=sorted(counts1,key=counts1.get,reverse=True)
  rev_dict={i:j for i,j in enumerate(vocabulary)}
  dict2={j:i for i,j in enumerate(vocabulary)}
  return rev_dict,dict2

In [0]:
index2word,word2index=dict_create(shortlisted_words)  # ranking each word on basis of its frequency with higher frequency are rated above

In [0]:
len(index2word)

53721

In [0]:
## Subsampling :- Approach to deal with the words with less frequency in the text based on probability threshold we will decide whether to keep stopwords or no

In [0]:
### """Creating the threshold and performing the subsampling"""

In [0]:
index_corpus=[word2index[w] for w in shortlisted_words] #converting text corpus to indexes to be fed into NN

In [0]:
index_corpus[:10]

In [0]:
c1=collections.Counter(shortlisted_words)

In [0]:
len(index_corpus)

16616688

In [0]:
len(index_corpus)
print(len(c1))#before feeding the data into model we need to convert data into index sequence

53721


In [0]:
index_corpus[:100]

In [0]:
thresh=0.00005
wordindex_counts=collections.Counter(index_corpus)

In [0]:
total_counts=len(index_corpus) #total number of unique words 
total_counts

16616688

In [0]:
wordindex_counts    #dictionary of rankindex :counts  

Counter({5233: 303,
         3080: 572,
         11: 131815,
         5: 325873,
         194: 7219,
         1: 593677,
         3133: 563,
         45: 28810,
         58: 22737,
         155: 8432,
         127: 10172,
         741: 2271,
         476: 3412,
         10571: 116,
         133: 9633,
         0: 1061396,
         27349: 25,
         102: 11868,
         854: 2029,
         2: 416629,
         15067: 68,
         150: 8736,
         3580: 481,
         10: 183153,
         190: 7378,
         4: 372201,
         10712: 114,
         214: 6433,
         6: 316376,
         1324: 1352,
         104: 11803,
         454: 3502,
         19: 109510,
         2731: 653,
         362: 4165,
         3672: 466,
         708: 2374,
         371: 4067,
         26: 73334,
         40: 37866,
         36: 44358,
         53: 25383,
         539: 3043,
         97: 12445,
         1423: 1254,
         2757: 646,
         18: 111831,
         567: 2879,
         686: 2449,
        

In [0]:
len(wordindex_counts)

53721

In [0]:
freq={word:count/total_counts for word,count in wordindex_counts.items()}  #dictionary of index :probability

In [0]:
p_drop={w:1-np.sqrt(thresh/freq[w]) for w in freq}

In [0]:
c1freq={word:count/total_counts for word,count in c1.items()}  
cp1_drop={w:1-np.sqrt(thresh/c1freq[w]) for w in c1freq}

In [0]:
sorted(cp1_drop.items(),key=lambda x:x[1],reverse=True)[:700]

In [0]:
for i in range(5):
  print(p_drop[i])   # As we can see for high freq words probability of drop is really high

In [0]:
len(p_drop)  #dictionary of word : probability

In [0]:
len(index_corpus)

In [0]:
train_words=[w for w in index_corpus if p_drop[w]< 0.6] #remember to preserve the order of words during the whole context

#period is taken as a token too
#probability is a hyperparameter   # 0.6 value should be taken by watching dictionary of words based on pdrop and judging the stop words

In [0]:
len(train_words)   # for removing the stop words  we got from 16 million to 7 million

In [0]:
[index2word[i] for i in train_words[:100]]

In [0]:
[index2word[i] for i in index_corpus[:150]]

In [0]:
def skipG_set_generation(batch,batch_index,word_window):
  
  random=word_window
  #print(random)
  if int((batch_index-random))> 0:
    wordstart=batch_index-random
  else:
    wordstart=0
  
  if int(batch_index+random) < len(batch):
    wordstop=batch_index+random
  else:
    wordstop=len(batch)-1
  
  window_target=set(batch[wordstart:batch_index]+batch[batch_index+1:wordstop+1])
  return list(window_target)


In [0]:
text='my name is puneet wassup what are your name'.split(" ")



In [0]:
skipG_set_generation(text,4,4)

In [0]:
shortlisted_words[:30]

In [0]:
train_words[:100]

In [0]:
#Train data generation
def skipg_batch_creation(shortwords,batchlength,wordwindow):
  batchcount=len(shortwords)//batchlength
  shortwords=shortwords[:batchcount*batchlength]
  for wordindex in range(0,len(shortwords),batchlength):
    inputwords=[]
    labelwords=[]
    wordbatch=shortwords[wordindex:wordindex+batchlength]
    for index in range(len(wordbatch)):
      batch_input=wordbatch[index]
      batchlabel=skipG_set_generation(wordbatch,index,wordwindow)
      labelwords.extend(batchlabel)
      inputwords.extend([batch_input]*len(batchlabel))
      yield inputwords,labelwords

In [0]:
a=skipg_batch_creation(train_words,10,2)

In [0]:
x,y=next(a)

In [0]:
x

In [0]:
y

In [0]:
input=tf.placeholder(tf.int32,[None])
label=tf.placeholder(tf.int32,[None,None])
word_embed=tf.Variable(tf.random_uniform((len(index2word),300),-1.0,1.0))
embedding=tf.nn.embedding_lookup(word_embed,input)

In [0]:
vocab_size=len(index2word)    

In [0]:
vocab_size  # total number of unique words  or total number of different classes

In [0]:
W1=tf.Variable(tf.truncated_normal((vocab_size,300),stddev=0.1))   # shd be of shape nclasses X dims
b1=tf.Variable(tf.zeros(vocab_size))

In [0]:
#  https://arxiv.org/pdf/1412.2007v2.pdf    very important paper to learn about sampled softmax which is the taking care of long softmax problem to scale up the model

In [0]:
# https://www.tensorflow.org/extras/candidate_sampling.pdf

# http://ruder.io/word-embeddings-softmax/index.html#negativesampling  important link for various softmax incase vocabsize is really big

In [0]:
loss_fn=tf.nn.sampled_softmax_loss(weights=W1,biases=b1,labels=label,inputs=embedding,num_sampled=100,num_classes=vocab_size)

In [0]:
cost_fn=tf.reduce_mean(loss_fn)
optim=tf.train.AdamOptimizer().minimize(cost_fn)

In [0]:
val_cnt=16
val_dict=100


In [0]:
epochs=2
batchlen=100
word_window=10

with tf.Session() as s:
  it=1
  loss=0
  s.run(tf.global_variables_initializer())
  
  for e in range(1,epochs+1):
    batches=skipg_batch_creation(train_words,batchlen,word_window)
    start=time.time()
    for x,y in batches:
      
      trainloss,_=s.run([cost_fn,optim],feed_dict={input:np.array(x),label:np.array(y)[:,None]})
      loss+=trainloss
     
      if (it)%50==0:
        print('trainloss:-',trainloss,' and epoch is:-',e)
      
      it+=1
        